# Solution 1

In this solution I will use dict approach, so I will go through all data and assign for each named entity ans store it class. To enhance my solution, I will take dataset from huggingface, because it contains more samples than files from coda lab. Also, I should remove all sentences from codalab test data, because I don't want to fit my model on test data

In [158]:
import json
from datasets import load_dataset
import re

In [82]:
def read_data(path: str) -> list[dict]:
    data_array = []
    with open(path, 'r') as file:
        for line in file:
            data = json.loads(line)
            data_array.append(data)
    return data_array

In [83]:
train = read_data('train.jsonl')
test = read_data('test.jsonl')
dataset_codalab = {'train': train, 'test': test}

In [84]:
dataset_hf = load_dataset("iluvvatar/RuNNE")

/home/bot_env/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for iluvvatar/RuNNE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/iluvvatar/RuNNE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Create train dataset

In [144]:
train_data = [entry for entry in dataset_codalab['train']]

In [145]:
def compare_and_take(source_dataset, target_dataset=dataset_codalab['test']) -> list[dict]:
    """
    Compares two datasets and returns a list of samples which is not lies in test_data
    Parameters:
        - source_dataset (list): A list that can contain samples from test dataset
        - target_dataset (list): A test dataset
    Returns:
        - output (list): A list of dictionaries containing sentences and named entities that are not similar between the two datasets.
    """
    output = []

    for source in source_dataset:
        flag = True
        for target in target_dataset:
            source_sentence = source['text']
            target_sentence = target['senences']
            if len(source_sentence) == len(target_sentence):
                same_chars = sum([source_char == target_char for source_char, target_char in zip(source_sentence, target_sentence)])
                
                if same_chars / len(source_sentence) > 0.5:
                    flag = False
                    break
        if flag:
            ners = []
            
            for ner in  source['entities']:
                start, end, cls = ner.split()
                ners.append([int(start), int(end) - 1, cls])
            output.append({'sentences': source_sentence, 'ners': ners})
    return output

In [146]:
train_data += compare_and_take(dataset_hf['test'])
train_data += compare_and_take(dataset_hf['train'])

## Fill the solution dictionary

In [151]:
solution_dict = dict()

In [152]:
for entry in train_data:
    sentences = entry['sentences'].lower()
    ners = entry['ners']

    for ner in ners:
        solution_dict[sentences[ner[0]: ner[1] + 1]] = ner[2]

In [153]:
len(solution_dict)

21221

## Testing

In [182]:
answer = []

for entry in dataset_codalab["test"]:
    ners = []
    prev_ind = 0
    for entity in solution_dict.keys():
        if entry["senences"].lower().find(entity) != -1:
            indexes = [index.start() for index in re.finditer(entity, entry["senences"].lower())]  # finding all occurrences of the entity into sentence
            for index in indexes:
                ners.append([index, index + len(entity) - 1, solution_dict[entity]])
    ners = sorted(ners, key=lambda x: x[0])
    answer.append({"ners": ners, "id": entry["id"]})

In [180]:
with open('output_data/test.jsonl', 'w') as outfile:
    for entry in answer:
        json.dump(entry, outfile)
        outfile.write('\n')